In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-recommendation-dataset/Ratings.csv
/kaggle/input/book-recommendation-dataset/Users.csv
/kaggle/input/book-recommendation-dataset/classicRec.png
/kaggle/input/book-recommendation-dataset/Books.csv
/kaggle/input/book-recommendation-dataset/DeepRec.png
/kaggle/input/book-recommendation-dataset/recsys_taxonomy2.png


In [2]:
import numpy as np
import pandas as pd

In [3]:
books = pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')
users = pd.read_csv('/kaggle/input/book-recommendation-dataset/Users.csv')
ratings = pd.read_csv('/kaggle/input/book-recommendation-dataset/Ratings.csv')

/tmp/ipykernel_18/2910231406.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('/kaggle/input/book-recommendation-dataset/Books.csv')


In [4]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


# Checking for Null Values

In [7]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

We won't be needing age, as there aren't enough Variables, and most of the Values are missing


In [8]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [9]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            2
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

# Checking for Duplicates

In [10]:
users.duplicated().sum()

0

In [11]:
ratings.duplicated().sum()

0

In [12]:
books.duplicated().sum()

0

# Let's Combine the Ratings and Books datasets, so that we can later on analyse the relation between the books and ratings with users

In [13]:
ratings_with_name = ratings.merge(books,on='ISBN')

Collaborative filtering based Recommendation System

Filtering out those users, who have Given ratings to books more than 150 Times

In [14]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 150
imp_users = x[x].index

In [15]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(imp_users)]

In [16]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [17]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

Creating Pivot Table of UsersID vs Books and values are Ratings.

In [18]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [19]:
pt.fillna(0,inplace=True)

In [20]:
pt

User-ID,254,1733,1903,2033,2110,2276,2766,2891,2977,3363,...,274301,274308,274808,275970,276680,277427,277478,277639,278188,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Using cosine_similarity to calculate the distance of the values of one book with another in a 2-D space (euclidean distance)

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
similarity_scores = cosine_similarity(pt)

In [23]:
similarity_scores.shape

(868, 868)

Got the Distance of Each Book with Respect to the other

In [24]:
similarity_scores

array([[1.        , 0.07980265, 0.00974903, ..., 0.09658948, 0.06448266,
        0.03549567],
       [0.07980265, 1.        , 0.25233692, ..., 0.06000464, 0.14873547,
        0.1435961 ],
       [0.00974903, 0.25233692, 1.        , ..., 0.03769791, 0.04493654,
        0.09205667],
       ...,
       [0.09658948, 0.06000464, 0.03769791, ..., 1.        , 0.06439035,
        0.01670749],
       [0.06448266, 0.14873547, 0.04493654, ..., 0.06439035, 1.        ,
        0.09936127],
       [0.03549567, 0.1435961 , 0.09205667, ..., 0.01670749, 0.09936127,
        1.        ]])

Fecting the Indexes of Books which are at the closest distance to the Book(input)

In [25]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:11]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-L'].values))
        
        data.append(item)
    
    return data

In [26]:
recommend('Lord of the Flies')

[['1984',
  'George Orwell',
  'http://images.amazon.com/images/P/0451524934.01.LZZZZZZZ.jpg'],
 ['Midnight in the Garden of Good and Evil',
  'John Berendt',
  'http://images.amazon.com/images/P/0679751521.01.LZZZZZZZ.jpg'],
 ['The Color Purple',
  'Alice Walker',
  'http://images.amazon.com/images/P/0671617028.01.LZZZZZZZ.jpg'],
 ['The Catcher in the Rye',
  'J.D. Salinger',
  'http://images.amazon.com/images/P/0316769487.01.LZZZZZZZ.jpg'],
 ['To Kill a Mockingbird',
  'Harper Lee',
  'http://images.amazon.com/images/P/0446310786.01.LZZZZZZZ.jpg'],
 ['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.LZZZZZZZ.jpg'],
 ['Misery',
  'Stephen King',
  'http://images.amazon.com/images/P/0451153553.01.LZZZZZZZ.jpg'],
 ['Into Thin Air : A Personal Account of the Mt. Everest Disaster',
  'JON KRAKAUER',
  'http://images.amazon.com/images/P/0385494785.01.LZZZZZZZ.jpg'],
 ['The Tao of Pooh',
  'Benjamin Hoff',
  'http://images.amazon.com/images/P/0140067477.01